# Прокофьев Игорь Витальевич
## Группа ДПИ22-1м
## Семинар 2, вариант 1

###Задание 1  

Напишите программу, которая будет создавать БД SQLite, создавать в ней таблицу  и заносить данные. Всего – 10 записей  
В конце работы нужно вывести данные из базы данных на экран.

Название БД – Univer  
Таблица - Students

| ID | Name | City   | Count_Stud | Adress | Founding_date |
| -- | ---- | -------| ----------: | ------ | ------------- |
| 1  | МГУ  | Москва | 10000      | Москва, Ленинские горы, д. 1 | 1775 |
| 2  | Финансовый Университет  | Москва | 7000      | Москва, Ленинградский проспект, 55 | 1919 |


In [1]:
# импортируем модуль субд, он будет использоваться также в следующих заданиях
import sqlite3


conn = sqlite3.connect('Univer.db')
cur = conn.cursor()

# почистим данные для избежания ошибок
cur.execute('DROP TABLE IF EXISTS Students;')
conn.commit()

# создаем таблицу в соответствии с заданием 1
cur.execute('''CREATE TABLE IF NOT EXISTS Students(
   ID INT PRIMARY KEY,
   Name TEXT,
   City TEXT,
   Count_Stud INTEGER,
   Adress TEXT,
   Founding_date INTEGER);
''')
conn.commit()

# Список данных для таблицы
univers_list = [('1', 'МГУ', 'Москва', '38150', 'Москва, Ленинские горы, д. 1', '1775'),
                ('2', 'Финансовый Университет', 'Москва', '19863', 'Москва, Ленинградский проспект, 55', '1919'),
                ('3', 'Алтайский государственный институт культуры', 'Барнаул', '1671', 'Алтайский край, Барнаул, ул. Юрина, д. 277', '1974'),
                ('4', 'Балтийский федеральный университет имени Иммануила Канта', 'Москва', '10616', 'Калининградская область, Калининград, ул. Александра Невского, д. 14', '1947'),
                ('5', 'Челябинский государственный университет', 'Челябинск', '15000', 'Челябинская область, Челябинск, ул. Братьев Кашириных, д. 129', '1976'),
                ('6', 'Сибирский государственный университет геосистем и технологий', 'Новосибирск', '5290', 'Новосибирская область, Новосибирск, ул. Плахотного, д. 10', '1933'),
                ('7', 'Казанский национальный исследовательский технологический университет', 'Казань', '18320', 'Республика Татарстан, Казань, ул. Карла Маркса, д. 68', '1890'),
                ('8', 'Московский государственный университет геодезии и картографии', ' Москва', '3078', 'Москва, Гороховский пер., д. 4', '1779'),
                ('9', 'Воронежский государственный университет', 'Воронеж', '18376', 'Воронежская область, Воронеж, Университетская пл., д. 1', '1918'),
                ('10', 'Пятигорский государственный университет', 'Пятигорск', '5500', 'Ставропольский край, Пятигорск, пр-т Калинина, д. 9', '1939')                         
]

# Записываем данные в таблицу
cur.executemany('INSERT INTO Students VALUES(?, ?, ?, ?, ?, ?);', univers_list)
conn.commit()

Для красивого вывода в stdout результатов SQL-запроса напишем функцию на основе модуля для ascii таблиц  
Эта функция пригодится нам и в следующих заданиях

In [2]:
# Код для красиового вывода результатов SQL запроса на экран
# Используется модуль для ascii таблиц PrettyTable

from prettytable import PrettyTable
def print_table(cur):
    # Определяем шапку и данные.
    th = list(map(lambda x: x[0], cur.description))
    td_temp = cur.fetchall()
    td = []
    for value in td_temp:
        td.extend(value)

    columns = len(th)  # Подсчитаем кол-во столбцов на будущее.

    table = PrettyTable(th)  # Определяем таблицу.

    # Cкопируем список td, на случай если он будет использоваться в коде дальше.
    td_data = td[:]
    # Входим в цикл который заполняет нашу таблицу.
    # Цикл будет выполняться до тех пор пока у нас не кончатся данные
    # для заполнения строк таблицы (список td_data).
    while td_data:
        # Используя срез добавляем первые пять элементов в строку.
        # (columns = 5).
        table.add_row(td_data[:columns])
        # Используя срез переопределяем td_data так, чтобы он
        # больше не содержал первых 5 элементов.
        td_data = td_data[columns:]

    print(table)  # Печатаем таблицу

def print_ascii_table(database, table):
    conn = sqlite3.connect(database)
    cur = conn.cursor()
    cur.execute(f'SELECT * FROM {table}')
    conn.commit()

    print_table(cur)

In [3]:
# Выводим результат задания 1

conn = sqlite3.connect('Univer.db')
cur = conn.cursor()
cur.execute('SELECT * FROM Students;')
conn.commit()

print_table(cur)

+----+----------------------------------------------------------------------+-------------+------------+----------------------------------------------------------------------+---------------+
| ID |                                 Name                                 |     City    | Count_Stud |                                Adress                                | Founding_date |
+----+----------------------------------------------------------------------+-------------+------------+----------------------------------------------------------------------+---------------+
| 1  |                                 МГУ                                  |    Москва   |   38150    |                     Москва, Ленинские горы, д. 1                     |      1775     |
| 2  |                        Финансовый Университет                        |    Москва   |   19863    |                  Москва, Ленинградский проспект, 55                  |      1919     |
| 3  |             Алтайский государстве


###Задание 2  

Из файла addresses.csv считать данные, а затем – экспортировать их в базу данных SQLite.




In [4]:
# https://campus.fa.ru/mod/folder/download_folder.php?id=351804
# импортируем модуль csv, он будет использоваться также в следующих заданиях
import csv

def reading_files(file_name):
    with open(file_name, encoding='utf-8') as r_file:
        # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.reader(r_file, delimiter = ",")
        # Считывание данных из CSV файла
        return [tuple(row) for row in file_reader]

def prepare_heads(data, tbl):
    # в файле задания отсутствуют заголовки, но мы возьмем первую строку в их качестве
    # только заменим пробелы между словами символами '_'
    # а если имя колонки начинается на цифру, ставим перед ней символ '_'
    # также заменим все знаки пунктуации и прочие символы на '_'
    heads_init = ['_'.join(name.strip().split()) + ' TEXT' for name in data[0]]
    for i, col_name in enumerate(heads_init):
        if col_name[0].isdigit():
            heads_init[i] = '_' + col_name
        tmp = ''
        for smb in heads_init[i]:
            if smb.isalnum() or smb in ('_', ' '):
                tmp += smb
            else:
                tmp += '_'
        heads_init[i] = tmp
    heads_init = ', '.join(heads_init)

    #подготавливаем текст SQL запроса на создание таблицы
    stmt = f'CREATE TABLE IF NOT EXISTS {tbl}(ID INT PRIMARY KEY, {heads_init});'
    return stmt

def prepare_data(data, tbl):
    # готовим список со строками таблицы для внесения в базу данных
    rows = []
    i = 1
    for row in data[1:]:
        rows.append((str(i), ) + row)
        i += 1

    # подготавливаем текст SQL запроса на внесение данных
    num_col = len(data[0]) + 1
    qm = ['?'] * num_col
    stmt = f'INSERT INTO {tbl} VALUES(' + ', '.join(qm) + ');'

    return stmt, rows

def data_transfer(in_name, out_name, table_name):

    data = reading_files(in_name)

    # создаем базу данных
    conn = sqlite3.connect(out_name)
    cur = conn.cursor()

    # почистим данные для избежания ошибок
    cur.execute(f'DROP TABLE IF EXISTS {table_name};')
    conn.commit()

    # Таким образом создается таблица с таким же числом колонок как и в csv
    # только названия исправляются в соотсветствии с требованиями
    cur.execute(prepare_heads(data, table_name))
    conn.commit()

    # вносим значения в базу данных
    cur.executemany(*prepare_data(data, table_name))
    conn.commit()

In [5]:
# Переносим данные
data_transfer('addresses.csv', 'addresses.db', 'list')

# Выводим результат задания 2
print_ascii_table('addresses.db', 'list')

+----+-----------------------+----------+----------------------------------+-------------+-----+--------+
| ID |          John         |   Doe    |        _120_jefferson_st_        |  Riverside  |  NJ | _08075 |
+----+-----------------------+----------+----------------------------------+-------------+-----+--------+
| 1  |          Jack         | McGinnis |           220 hobo Av.           |    Phila    |  PA | 09119  |
| 2  |     John "Da Man"     |  Repici  |        120 Jefferson St.         |  Riverside  |  NJ | 08075  |
| 3  |        Stephen        |  Tyler   | 7452 Terrace "At the Plaza" road |   SomeTown  |  SD |  91234 |
| 4  |                       | Blankman |                                  |   SomeTown  |  SD |  00298 |
| 5  | Joan "the bone", Anne |   Jet    |       9th, at Terrace plc        | Desert City |  CO | 00123  |
+----+-----------------------+----------+----------------------------------+-------------+-----+--------+


###Задание 3  

Открыть таблицу, созданную в Задании 1.  
Выбрать из нее данные в соответствии с условием (университеты с количеством студентов больше 5000).  
Эти данные вывести на экран.  
Создать файл csv и сохранить выбранные данные в этот файл:  
Сделать заголовки «Название», «Город», «Количество студентов» (только 3 столбца). Сохранить в файле соответствующие данные.

In [6]:
# Выводим результат запроса согласно условию задания 3

conn = sqlite3.connect('Univer.db')
cur = conn.cursor()
cur.execute('SELECT * FROM Students WHERE Count_Stud > 5000;')
conn.commit()

print_table(cur)

+----+----------------------------------------------------------------------+-------------+------------+----------------------------------------------------------------------+---------------+
| ID |                                 Name                                 |     City    | Count_Stud |                                Adress                                | Founding_date |
+----+----------------------------------------------------------------------+-------------+------------+----------------------------------------------------------------------+---------------+
| 1  |                                 МГУ                                  |    Москва   |   38150    |                     Москва, Ленинские горы, д. 1                     |      1775     |
| 2  |                        Финансовый Университет                        |    Москва   |   19863    |                  Москва, Ленинградский проспект, 55                  |      1919     |
| 4  |       Балтийский федеральный унив

In [7]:
# Теперь уточняем предыдущий запрос (выбираем только 3 колонки)

cur.execute('''SELECT Name AS Название,
                    City AS Город,
                    Count_Stud AS "Количество студентов"
            FROM Students WHERE Count_Stud > 5000;
''')
conn.commit()

# Результат запроса запишем в список кортежей
my_query = [tuple(map(lambda x: x[0], cur.description))]
my_query.extend(cur.fetchall())


with open('students.csv', mode='w', encoding='utf-8') as w_file:
    # Создаем объект writer, указываем символ-разделитель ","
    file_writer = csv.writer(w_file, delimiter = ",", lineterminator="\r")
    for string in my_query:
        # Записываем в файл соотвествующие строки
        file_writer.writerow(string)


###Задание 4
Создать БД с одной таблицей – Автомобили(Марка, модель, Цвет, Цена). Создать файл csv с такой же структурой. Реализовать программу, которая спрашивает у пользователя соответствующие данные для ввода; эти строки нужно после ввода добавлять в БД и в файл csv. Ввод прекращается, когда пользователь введет слово «Конец».

In [8]:
# Определяем функцию воода данных пользователем
def cars_input():
    lst = []
    counter = 1
    while True:
        print(f'Внесение данных по автомобилю номер {counter}')
        print('Если ввод завершен, наберите слово "Конец"')
        brand = input('Введите марку: ')
        if brand.lower() == 'конец':
           print()
           break
        model = input('Введите модель: ')
        if model.lower() == 'конец':
           print()
           break
        color = input('Введите цвет: ')
        if color.lower() == 'конец':
           print()
           break
        while True:
            price = input('Введите цену (целое число): ')
            if price.lower() == 'конец':
                break
            if price.isdigit():
                break
        if price.lower() == 'конец':
            print()
            break
        lst.append((counter, brand, model, color, int(price)))
        counter += 1
        print()
    print('Ввод данных завершён.')
    return lst

In [9]:
#пользователь вводит данные
result = cars_input()

Внесение данных по автомобилю номер 1
Если ввод завершен, наберите слово "Конец"
Введите марку: йцук
Введите модель: фыва
Введите цвет: ячсм
Введите цену (целое число): 1234

Внесение данных по автомобилю номер 2
Если ввод завершен, наберите слово "Конец"
Введите марку: ъхзщ
Введите модель: эждл
Введите цвет: .юбь
Введите цену (целое число): 9876

Внесение данных по автомобилю номер 3
Если ввод завершен, наберите слово "Конец"
Введите марку: Конец

Ввод данных завершён.


In [10]:
# создаем базу данных
conn = sqlite3.connect('database.db')
cur = conn.cursor()

# чистим данные для избежания ошибок
cur.execute('DROP TABLE IF EXISTS cars;')
conn.commit()

# создаем таблицу в соответствии с заданием 4
cur.execute('''CREATE TABLE IF NOT EXISTS cars(
   ID INT PRIMARY KEY,
   brand TEXT,
   model TEXT,
   color TEXT,
   price INTEGER);
''')
conn.commit()

# вносим значения в таблицу
cur.executemany('INSERT INTO cars VALUES(?, ?, ?, ?, ?);', result)
conn.commit()

# проверяем всё ли внесено без ошибок
cur.execute('SELECT * FROM cars;')
conn.commit()
print_table(cur)

+----+-------+-------+-------+-------+
| ID | brand | model | color | price |
+----+-------+-------+-------+-------+
| 1  |  йцук |  фыва |  ячсм |  1234 |
| 2  |  ъхзщ |  эждл |  .юбь |  9876 |
+----+-------+-------+-------+-------+


In [11]:
# также записываем таблицу в csv файл
with open('cars.csv', mode='w', encoding='utf-8') as w_file:
    # Создаем объект writer, указываем символ-разделитель ","
    file_writer = csv.writer(w_file, delimiter = ",", lineterminator="\r")
    file_writer.writerow(('ID', 'brand', 'model', 'color', 'price'))
    for string in result:
        # Записываем в файл соотвествующие строки
        file_writer.writerow(string)

###Задание 5
Для csv-файла реализовать:  
* копирование файла в БД только строк, указанных пользователем;
* копирование файла в БД, кроме строк, указанных пользователем, но вместо их будет заноситься текст «null»;


In [12]:
def cond_input():
    while True:
        cond_raw = input('Введите номера строк, которые хотите включить в базу: ')
        cond = cond_raw.split()
        flag = False
        for number in cond:
            if not number.isdigit():
                flag = True
        if flag == False:
            cond = [int(i) for i in cond]
            return set(cond)
        print('Необходимо вводить только целые числа через пробел')

def prepare_data_incl(data, tbl, cond):
    # готовим список со строками таблицы для внесения в базу данных
    rows = []
    i = 1
    for row in data[1:]:
        if i in cond:
            rows.append((str(i), ) + row)
        i += 1

    # подготавливаем текст SQL запроса на внесение данных
    num_col = len(data[0]) + 1
    qm = ['?'] * num_col
    stmt = f'INSERT INTO {tbl} VALUES(' + ', '.join(qm) + ');'

    return stmt, rows

def prepare_data_not_incl(data, tbl, cond):
    # готовим список со строками таблицы для внесения в базу данных
    rows = []
    i = 1
    for row in data[1:]:
        if i not in cond:
            rows.append((str(i), ) + row)
        else:
            rows.append((str(i), ) + ('NULL', ) * len(row))
        i += 1

    # подготавливаем текст SQL запроса на внесение данных
    num_col = len(data[0]) + 1
    qm = ['?'] * num_col
    stmt = f'INSERT INTO {tbl} VALUES(' + ', '.join(qm) + ');'

    return stmt, rows

def data_transfer_cond(in_name, out_name, table_name, prepare_foo, cond):

    data = reading_files(in_name)

    # создаем базу данных
    conn = sqlite3.connect(out_name)
    cur = conn.cursor()

    # почистим данные для избежания ошибок
    cur.execute(f'DROP TABLE IF EXISTS {table_name};')
    conn.commit()

    # Таким образом создается таблица с таким же числом колонок как и в csv
    # только названия исправляются в соотсветствии с требованиями
    cur.execute(prepare_heads(data, table_name))
    conn.commit()

    # вносим значения в базу данных
    cur.executemany(*prepare_foo(data, table_name, cond))
    conn.commit()

In [13]:
cond = cond_input()

Введите номера строк, которые хотите включить в базу: 2 5


In [14]:
# Переносим данные, только строки, указанные пользователем
data_transfer_cond('addresses.csv', 'database_2.db', 'cars', prepare_data_incl, cond)
# Выводим результат задания 5.1
print_ascii_table('database_2.db', 'cars')

+----+-----------------------+--------+---------------------+-------------+-----+--------+
| ID |          John         |  Doe   |  _120_jefferson_st_ |  Riverside  |  NJ | _08075 |
+----+-----------------------+--------+---------------------+-------------+-----+--------+
| 2  |     John "Da Man"     | Repici |  120 Jefferson St.  |  Riverside  |  NJ | 08075  |
| 5  | Joan "the bone", Anne |  Jet   | 9th, at Terrace plc | Desert City |  CO | 00123  |
+----+-----------------------+--------+---------------------+-------------+-----+--------+


In [15]:
# Переносим данные, кроме строк, указанных пользователем (заполняем их значением NULL)
data_transfer_cond('addresses.csv', 'database_2.db', 'cars', prepare_data_not_incl, cond)
# Выводим результат задания 5.2
print_ascii_table('database_2.db', 'cars')

+----+---------+----------+----------------------------------+-----------+------+--------+
| ID |   John  |   Doe    |        _120_jefferson_st_        | Riverside |  NJ  | _08075 |
+----+---------+----------+----------------------------------+-----------+------+--------+
| 1  |   Jack  | McGinnis |           220 hobo Av.           |   Phila   |  PA  | 09119  |
| 2  |   NULL  |   NULL   |               NULL               |    NULL   | NULL |  NULL  |
| 3  | Stephen |  Tyler   | 7452 Terrace "At the Plaza" road |  SomeTown |  SD  |  91234 |
| 4  |         | Blankman |                                  |  SomeTown |  SD  |  00298 |
| 5  |   NULL  |   NULL   |               NULL               |    NULL   | NULL |  NULL  |
+----+---------+----------+----------------------------------+-----------+------+--------+
